<a href="https://colab.research.google.com/github/zamoravm1/ict-for-smart-mobility/blob/main/Lab2_SmartMobility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>ICT for Smart Mobility</center></h1>

<center><font size="5">Lab	#2	- Prediction	using	ARIMA	models </font></center>




Consider the time series of rentals in the previous steps, for each city. Consider a time period of 30 days for which you have data (pay attention to stationarity – are December or January 2017 good months?).

The goal of this lab is to experiment with predictions using ARIMA models, and to check how the error changes with respect to hyper-parameters. For this, you have to consider the various parameters in the 
ARIMA model training, including:

    • The model parameters (p,d,q)
    • The training windows size N (how many past samples are used for training)
    • The training policy, i.e., expanding versus sliding windows.

In [ ]:
# import section 
import pymongo
from datetime import datetime
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# creating a mongoDB Client and db 
client = pymongo.MongoClient('bigdatadb.polito.it:27017',
                     username='ictts',
                     password='Ict4SM22!',
                     authSource='carsharing',
                     authMechanism='SCRAM-SHA-1',
                     ssl=True,
                     tlsAllowInvalidCertificates=True)
db = client['carsharing']

In [ ]:
db

Database(MongoClient(host=['bigdatadb.polito.it:27017'], document_class=dict, tz_aware=False, connect=True, authsource='carsharing', authmechanism='SCRAM-SHA-1', tlsallowinvalidcertificates=True, tlsdisableocspendpointcheck=True, tls=True), 'carsharing')

## Data collection and preprocessing

In this study its aimed to predict the number of rentals per-hour 
in a given city, knowing the data collected in previous days. In order to build a time-series, its selected the data between October 1st 2017 and October 30th 2017. December and January, or even August in the case of Torino are avoid it because the behavior of the users is atypic so that data is not stationary as it is required for ARIMA models.

In [ ]:
# choose the interval
start_date = datetime(2017,10,1)
start_unixtime = time.mktime(start_date.timetuple())
# start_date_unix = 1506816000.0
final_date = datetime(2017,10,31)
final_unixtime = time.mktime(final_date.timetuple())
# start_date_unix = 1509408000.0

### Task 1

For each city, consider the selected period of 30 days. Extract the number of rentals recorded at each 
hour – after proper filtering the outliers and those bookings that are not rentals.

In [ ]:
#Creating dbs variables

# Car2Go
PermanentBookings = db["PermanentBookings"]
PermanentParkings = db["PermanentParkings"]
# Enjoy
enjoy_PermanentBookings = db["enjoy_PermanentBookings"]
enjoy_PermanentParkings = db["enjoy_PermanentParkings"]

In [ ]:

# Booking data per city

dfs = []
# BOOKINGS DF
Torino_Bookings = PermanentBookings.aggregate([{'$match': {'city': 'Torino', 'init_time': { '$gte': start_unixtime, '$lte': final_unixtime } }},{'$project': {'city': 1, 'timestamp': '$init_time', 'duration': { '$divide': [ { '$subtract': ["$final_time", "$init_time"] }, 60 ] }, 'origin': {'$arrayElemAt' : ['$origin_destination.coordinates', 0]}, 'destination': {'$arrayElemAt' : ['$origin_destination.coordinates', 1]}}}])
df_bookingsTorino = pd.DataFrame(Torino_Bookings)
dfs.append(('Bookings Torino', df_bookingsTorino))

enjoy_Torino_Bookings = enjoy_PermanentBookings.aggregate([{'$match': {'city': 'Torino', 'init_time': { '$gte': start_unixtime, '$lte': final_unixtime } }},{'$project': {'city': 1, 'timestamp': '$init_time', 'duration': { '$divide': [ { '$subtract': ["$final_time", "$init_time"] }, 60 ] }, 'origin': {'$arrayElemAt' : ['$origin_destination.coordinates', 0]}, 'destination': {'$arrayElemAt' : ['$origin_destination.coordinates', 1]}}}])
df_bookingsEnjoyTorino = pd.DataFrame(enjoy_Torino_Bookings)
dfs.append(('Bookings Torino Enjoy', df_bookingsEnjoyTorino))

Denver_Bookings = PermanentBookings.aggregate([{'$match': {'city': 'Denver', 'init_time': { '$gte': start_unixtime, '$lte': final_unixtime } }},{'$project': {'city': 1, 'timestamp': '$init_time', 'duration': { '$divide': [ { '$subtract': ["$final_time", "$init_time"] }, 60 ] }, 'origin': {'$arrayElemAt' : ['$origin_destination.coordinates', 0]}, 'destination': {'$arrayElemAt' : ['$origin_destination.coordinates', 1]}}}])
df_bookingsDenver = pd.DataFrame(Denver_Bookings)
dfs.append(('Bookings Denver', df_bookingsDenver))

In [ ]:
# Filtering outliers

# Booking time
min_duration=5
max_duration=180
# False rental
#{'$match': {'moved':  True, 'duration':{'$gte': min_duration,'$lte': max_duration}}}

# BOOKINGS DF FILTERED
dfs_clean = []

Torino_Bookings_filter=PermanentBookings.aggregate([{'$match': {'city': 'Torino', 'init_time': { '$gte': start_unixtime, '$lte': final_unixtime } }},{'$project': {'city': 1, 'timestamp': '$init_time', 'duration': { '$divide': [ { '$subtract': ["$final_time", "$init_time"] }, 60 ] }, 'moved': {'$ne': [{'$arrayElemAt' : ['$origin_destination.coordinates', 0]},{'$arrayElemAt' : ['$origin_destination.coordinates', 1]}]}}},{'$match': {'moved':  True, 'duration':{'$gte': min_duration,'$lte': max_duration}}}])
df_bookingsTorino_filter = pd.DataFrame(Torino_Bookings_filter)
dfs_clean.append(('Bookings Torino filtered',df_bookingsTorino_filter))

enjoy_Torino_Bookings_filter = enjoy_PermanentBookings.aggregate([{'$match': {'city': 'Torino', 'init_time': { '$gte': start_unixtime, '$lte': final_unixtime } }},{'$project': {'city': 1, 'timestamp': '$init_time', 'duration': { '$divide': [ { '$subtract': ["$final_time", "$init_time"] }, 60 ] },'moved': {'$ne': [{'$arrayElemAt' : ['$origin_destination.coordinates', 0]},{'$arrayElemAt' : ['$origin_destination.coordinates', 1]}]}}},{'$match': {'moved':  True, 'duration':{'$gte': min_duration,'$lte': max_duration}}}])
df_bookingsEnjoyTorino_filter = pd.DataFrame(enjoy_Torino_Bookings_filter)
dfs_clean.append(('Bookings Torino Enjoy filtered', df_bookingsEnjoyTorino_filter))

Denver_Bookings_filter = PermanentBookings.aggregate([{'$match': {'city': 'Denver', 'init_time': { '$gte': start_unixtime, '$lte': final_unixtime } }},{'$project': {'city': 1, 'timestamp': '$init_time', 'duration': { '$divide': [ { '$subtract': ["$final_time", "$init_time"] }, 60 ] },'moved': {'$ne': [{'$arrayElemAt' : ['$origin_destination.coordinates', 0]},{'$arrayElemAt' : ['$origin_destination.coordinates', 1]}]}}},{'$match': {'moved':  True, 'duration':{'$gte': min_duration,'$lte': max_duration}}}])
df_bookingsDenver_filter = pd.DataFrame(Denver_Bookings_filter)
dfs_clean.append(('Bookings Denver filtered', df_bookingsDenver_filter))

In [ ]:
# Separate per hours
occurences_torino=[]
occurences_torinoenjoy=[]
occurences_denver=[]
hours=[]

for hour in range(24):
    hours.append("0"+str(hour)+":00" if hour < 10 else str(hour)+":00")
    occurences_torino.append(dfs_clean[0][1]['timestamp'] , len(dfs_clean[0][1].loc[(dfs_clean[0][1]['timestamp'] > start_unixtime + hour*3600) & (dfs_clean[0][1]['timestamp'] <= start_unixtime + (hour + 1)*3600)].values))
    occurences_torinoenjoy.append(len(dfs_clean[1][1].loc[(dfs_clean[1][1]['timestamp'] > start_unixtime + hour*3600) & (dfs_clean[1][1]['timestamp'] <= start_unixtime + (hour + 1)*3600)].values))
    occurences_denver.append(len(dfs_clean[0][1].loc[(dfs_clean[0][1]['timestamp'] > start_unixtime + hour*3600) & (dfs_clean[0][1]['timestamp'] <= start_unixtime + (hour + 1)*3600)].values))

In [ ]:
occurences_torino

[58,
 31,
 21,
 22,
 26,
 22,
 35,
 46,
 76,
 87,
 112,
 82,
 86,
 86,
 103,
 123,
 105,
 113,
 85,
 78,
 64,
 67,
 33,
 17]

### Task 2

Check that there are no missing samples (recall – ARIMA models assume a regular time series, with no missing data). In case of missing samples – define a policy for fitting missing data. For instance, use i) the last value, or ii) the average value, or iii) replace with zeros, or iv) replace with average
value for the given time bin, etc.

In [ ]:
dfs[0][1]['timestamp'][0]

24